<a href="https://colab.research.google.com/github/PraeJi/OCR_with_Qwen2VLModel/blob/main/OCR_qwen_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture

!pip install git+https://github.com/huggingface/transformers
!pip install torch
!pip install accelerate
!pip install Pillow

qwen 2 (ใช้ได้ *แต่ไม่แม่น*)

In [ ]:
import gradio as gr
from PIL import Image
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor

# -------------------------------------------------------------------
# Setup device
# -------------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------------------------------------------------------
# Load model & processor
# -------------------------------------------------------------------
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype=torch.float16,       # Reduce GPU memory
    device_map=None,                 # Disable auto map (we assign manually)
).to(device)                         # Move model to GPU

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

Unrecognized keys in `rope_parameters` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

In [ ]:

# -------------------------------------------------------------------
# OCR function
# -------------------------------------------------------------------
def extract_text_from_image(image):

    # Convert PIL image to RGB
    # image = image.convert("RGB")

    # Chat message
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": "Extract text from the image (OCR)"}
            ]
        }
    ]

    text_prompt = processor.apply_chat_template(
        messages,
        add_generation_prompt=True
    )

    # Processor → tensors (still on CPU)
    inputs = processor(
        text=[text_prompt],
        images=[image],
        padding=True,
        return_tensors="pt"
    )

    # -------------------------------------------------------------------
    # Move all input tensors to GPU
    # -------------------------------------------------------------------
    for k, v in inputs.items():
        inputs[k] = v.to(device)

    # -------------------------------------------------------------------
    # Generate with mixed precision (ลด VRAM)
    # -------------------------------------------------------------------
    with torch.autocast(device_type="cuda", dtype=torch.float16):
        output_ids = model.generate(
            **inputs,
            max_new_tokens=512,
        )

    # Decode
    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(inputs["input_ids"], output_ids)
    ]

    extracted_text = processor.batch_decode(
        generated_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )[0]

    return extracted_text


In [ ]:
image_path = "/content/Screenshot 2025-11-29 212223.png"

result = extract_text_from_image(image_path)
result

"Hello! I'm Jirapa (Prae)\n\nAI Engineer | Computer Science.\n\nPassionate about building and learning in the fields of Machine Learning and Deep Learning.\n\nBased in Thailand.\nBackground in Computer Science.\nMy Study Notes: MyStudyNotes\ncontact: jirapa.official@gmail.com\nAlways learning & exploring new technologies"